In [37]:
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

In [38]:
URL = 'https://hh.ru'
profession = 'Программист'

service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(URL)
driver.implicitly_wait(1)



# headers = {
#     'Accept': '*/*',
#     'User-Agent': 'Custom'
# }
#
# req = requests.get(url, headers={'User-Agent': 'Custom'})



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\dm1tr\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [41]:
src = driver.page_source
soup = BeautifulSoup(src, 'lxml')

In [42]:
main_block = soup.find_all('div', class_='dashboard-tiles-item__body')

for b in main_block:
    print(b)
    print()

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Вакансии дня</div><div class="dashboard-tiles-item__salary">50 000 – 75 000 руб.</div></div><div class="dashboard-tiles-item__counter"><span>14 вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Компании дня</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>1913<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Работа из дома</div><div class="dashboard-tiles-item__salary"></div></div><div class="dashboard-tiles-item__counter"><span>37260<!-- --> <!-- -->вакансий</span></div></div>

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles

In [44]:
for sub in main_block:
    if sub.find('div', class_='dashboard-tiles-item__title').text == profession:
        profession_selected_block = sub
print(profession_selected_block)

<div class="dashboard-tiles-item__body"><div class="dashboard-tiles-item__content"><div class="dashboard-tiles-item__title">Программист</div><div class="dashboard-tiles-item__salary">до <!-- -->515 000<!-- --> <!-- -->руб.</div></div><div class="dashboard-tiles-item__counter"><span>18323<!-- --> <!-- -->вакансии</span></div></div>
